In [1]:
import ase
from ase import io
from ase.io.espresso import write_espresso_in
import numpy as np
import os
from ase.visualize import view

In [ ]:
# Read the slab file into an ASE Atoms object
slab_original = ase.io.read('../zrb2-slab-001/qe-calcs/relax-144atoms/zrb2-slab-001-relax.out', index=-1)
# view(slab)

In [4]:
# Input setting for Quantum ESPRESSO
pseudopotentials = {'Zr': 'Zr.pbe-spn-kjpaw_psl.1.0.0.UPF',
                    'B' : 'B.pbe-n-kjpaw_psl.1.0.0.UPF',
                    'O' : 'O.pbe-n-kjpaw_psl.0.1.UPF'}
input_data = {
    'scf': {
        'calculation': 'scf',
        'outdir': './tmp',
        'pseudo_dir': '/home/cganley2/qe-data/pseudopotentials/SSSP_1.3.0_PBE_precision', # will need to update this--do you have this directory somewhere?
        'tprnfor': True,
        'tstress': True,
        'ecutwfc': 50,
        'ecutrho': 400,
        'electron_maxstep': 200,
        'occupations': 'smearing',
        'conv_thr': 1e-7,
        'mixing_mode': 'local-TF',
        'ion_dynamics': 'bfgs',
    },
    'relax': {
        'calculation': 'relax',
        'outdir': './tmp',
        'pseudo_dir': '/home/cganley2/qe-data/pseudopotentials/SSSP_1.3.0_PBE_precision', # same as above
        'tprnfor': True,
        'tstress': True,
        'ecutwfc': 50,
        'ecutrho': 400,
        'electron_maxstep': 200,
        'occupations': 'smearing',
        'conv_thr': 1e-7,
        'mixing_mode': 'local-TF',
        'ion_dynamics': 'bfgs',
    }
}
kpts = (2, 2, 1)

In [ ]:
# iterate over unique z-values in slab file
# randomly change up to 4 atoms to O
# create input file for SCF and relax
# need to consider output directory parsing

In [5]:
unique_z_vals = np.unique(np.round(slab.get_positions(), 1)[:, 2])
unique_z_vals

array([10.1, 11.8, 13.6, 15.3, 17.1, 18.8])

In [ ]:
slab = slab_original.copy()
for z_val in unique_z_vals:
    # find the indices of the atoms in the slab that have this z value
    layer_indices = np.array([])
    for index, xyz_coords in enumerate(slab.get_positions()):
        if np.round(xyz_coords[2], 1) == z_val:
            # print('The rounded z coordinate matches the z value in the loop. The atom index is {0} and is atom {1}'.format(index, slab.get_chemical_symbols()[index]))
            layer_indices = np.append(layer_indices, index)
    
    # identify 1 to 4 random indices from the layer to use for defecting
    n_subs = np.random.randint(1, min(4, len(layer_indices)) + 1)
    np.random.shuffle(layer_indices)
    defect_indices = layer_indices[:n_subs]
    
    # substitute an O at each index, save the structure and QE input file
    for index in defect_indices:
        symbols = slab.get_chemical_symbols()
        symbols[int(index)] = 'O' # Replaces atom at the index with Oxygen
        slab.set_chemical_symbols(symbols)
        
    layer_N = np.where(unique_z_vals == z_val)[0][0] # get layer number for file name
    scf_input = os.path.join("zrb2-001/qe-calcs", f"layer{layer_N + 1}-o{n_subs}", "scf", "pw.in")
    relax_input = os.path.join("zrb2-001/qe-calcs", f"layer{layer_N + 1}-o{n_subs}", "relax", "pw.in")

    # Makes directories if not already created
    os.makedirs(os.path.dirname(scf_input), exist_ok=True)
    os.makedirs(os.path.dirname(relax_input), exist_ok=True)
        
    # Adds input files
    with open(scf_input, "w") as f:
        write_espresso_in(f, slab, input_data=input_data['scf'], pseudopotentials=pseudopotentials, kpts=kpts)
    with open(relax_input, "w") as f:
        write_espresso_in(f, slab, input_data=input_data['relax'], pseudopotentials=pseudopotentials, kpts=kpts)